#yourNAME, yourID

# Imports

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

# LuffyDodge (3)

In [ ]:
class LuffyDodgeEnv(gym.Env):
    """
    Custom environment where Luffy must dodge cannonballs.
    Luffy can move left, right, or stay still to avoid being hit.
    """

    metadata = {"render_modes": ["human"]}

    def __init__(self, render_mode=None):
        """
        Initialize environment parameters.
        """
        super(LuffyDodgeEnv, self).__init__()

        # Actions: 0 = left, 1 = stay, 2 = right
        self.action_space = spaces.Discrete(3)

        # Observation: [Luffy_x, Cannonball_x, Cannonball_y]
        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0], dtype=np.float32),
            high=np.array([9, 9, 9], dtype=np.float32),
            dtype=np.float32
        )

        self.render_mode = render_mode
        self.reset()

    def reset(self, seed=None, options=None):
        """
        Reset the environment to its initial state.
        """
        super().reset(seed=seed)

        # Luffy starts in the middle bottom
        self.luffy_x = 5.0

        # Cannonball starts at a random x and top y = 9
        self.cannon_x = np.random.randint(0, 10)
        self.cannon_y = 9.0

        # Return initial observation
        obs = np.array([self.luffy_x, self.cannon_x, self.cannon_y], dtype=np.float32)
        return obs, {}

    def step(self, action):
        """
        Performs one time-step transition in the environment.

        Parameters:
            action (int): The action taken by Luffy.
                          0 = move left, 1 = stay still, 2 = move right

        Returns:
            observation (np.array): The next state [luffy_x, cannon_x, cannon_y]
            reward (float): +1 if survived this step, -10 if hit by cannonball
            done (bool): True if Luffy is hit, False otherwise
            truncated (bool): Optional flag if max steps are reached
            info (dict): Additional info (empty for now)

        Notes for Students:
        - Update Luffy’s position based on the action.
        - Move the cannonball down by one step.
        - Check for collision (same x and y == 0).
        - If collision occurs, assign -10 reward and mark `done = True`.
        - Otherwise, assign +1 reward and continue.
        - If the cannonball reaches y = 0 but misses Luffy, reset it to top at a random x position.
        """
       reward=0
       done=False
       if action=0:
            self.luffy_x = max(0, self.luffy_x - 1)
       elif action=1:
            pass
       elif action=2:
            self.luffy_x = min(9, self.luffy_x + 1)
       self.cannon_y -= 1
       if self.cannony==0:
            if self.luffy_x==self.cannon_x:
                reward=-10
                done=True
            else:
                reward=1
                self.cannon_x = np.random.randint(0, 10)
      obs=np.array([self.luffy_x, self.cannon_x, self.cannon_y], dtype=np.float32)
      return obs, reward, done, False, {}

    def render(self):
        """
        Render the environment as simple text output.
        """
        grid = np.full((10, 10), " ", dtype=str)
        grid[int(self.cannon_y), int(self.cannon_x)] = "O"
        grid[0, int(self.luffy_x)] = "L"
        print("\n".join(["".join(row) for row in grid[::-1]]))
        print("-" * 10)

    def close(self):
        """Close the environment."""
        pass

In [ ]:
env = LuffyDodgeEnv()
obs, _ = env.reset()

done = False
while not done:
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    env.render()
    print(f"Action: {action}, Reward: {reward}")

          
         O
          
          
          
          
          
          
          
     L    
----------
Action: 1, Reward: 1.0
          
          
         O
          
          
          
          
          
          
     L    
----------
Action: 1, Reward: 1.0
          
          
          
         O
          
          
          
          
          
    L     
----------
Action: 0, Reward: 1.0
          
          
          
          
         O
          
          
          
          
    L     
----------
Action: 1, Reward: 1.0
          
          
          
          
          
         O
          
          
          
     L    
----------
Action: 2, Reward: 1.0
          
          
          
          
          
          
         O
          
          
      L   
----------
Action: 2, Reward: 1.0
          
          
          
          
          
          
          
         O
          
     L    
----------
Action: 0, Rewa

# Policy Evaluation (2)

In [ ]:
# Grid size
GRID_SIZE = 5

# Actions: 0 = left, 1 = stay, 2 = right
ACTIONS = [0, 1, 2]
NUM_ACTIONS = len(ACTIONS)

# All possible states: (Luffy_x, Cannon_x, Cannon_y)
states = [(lx, cx, cy) for lx in range(GRID_SIZE)
                         for cx in range(GRID_SIZE)
                         for cy in range(GRID_SIZE)]
NUM_STATES = len(states)

# Initialize policy randomly
policy = np.random.choice(ACTIONS, size=NUM_STATES)

# Initialize value function
V = np.zeros(NUM_STATES)

# Discount factor
gamma = 0.9


def state_to_index(state):
    """Convert a (luffy_x, cannon_x, cannon_y) state tuple into its index in the state list."""
    lx, cx, cy = state
    return lx * GRID_SIZE * GRID_SIZE + cx * GRID_SIZE + cy


def transition(state, action):
    """
    Deterministic transition function for the environment.

    Parameters
    ----------
    state : tuple (luffy_x, cannon_x, cannon_y)
    action : int (0=left, 1=stay, 2=right)

    Returns
    -------
    next_state : tuple or None
        Next state after taking the action.
    reward : float
        Reward received after the transition.
    done : bool
        Whether the episode terminates (Luffy hit by cannonball).
    """
    lx, cx, cy = state

    # Move Luffy
    if action == 0:
        lx = max(0, lx - 1)
    elif action == 2:
        lx = min(GRID_SIZE - 1, lx + 1)

    # Move cannonball down
    cy -= 1

    # Check terminal condition
    if cy < 0:
        if lx == cx:
            # Hit
            return None, -10.0, True
        else:
            # Miss → reset cannonball to top
            return (lx, cx, GRID_SIZE - 1), +1.0, False

    # Otherwise just one step closer to bottom
    return (lx, cx, cy), +1.0, False


def policy_evaluation(policy, V, theta=1e-4):
    """
    Iteratively evaluate the value function for the given policy.
    """
    while True:
        delta = 0
        for s, state in enumerate(states):
            v = V[s]  # old value
            action = policy[s]

            next_state, reward, done = transition(state, action)

            if done or next_state is None:
                V[s] = reward
            else:
                V[s] = reward + gamma * V[state_to_index(next_state)]

            delta = max(delta, abs(v - V[s]))

        if delta < theta:
            break
    return V



def policy_improvement(V, policy):
    """Greedy policy improvement based on the current value function."""
    policy_stable = True
    for s, state in enumerate(states):
        old_action = policy[s]
        action_values = []

        # Try all actions and pick the best one
        for a in ACTIONS:
            next_state, reward, done = transition(state, a)
            if done:
                action_values.append(reward)
            else:
                action_values.append(reward + gamma * V[state_to_index(next_state)])

        best_action = np.argmax(action_values)
        policy[s] = best_action

        # Check if policy changed
        if old_action != best_action:
            policy_stable = False
    return policy, policy_stable


def policy_iteration():
    """Run the full policy iteration loop."""
    global V, policy
    iteration = 0
    while True:
        iteration += 1
        V = policy_evaluation(policy, V)
        policy, stable = policy_improvement(V, policy)
        print(f"Iteration {iteration} completed.")
        if stable:
            print("✅ Policy converged!")
            break
    return policy, V


if __name__ == "__main__":
    optimal_policy, optimal_V = policy_iteration()

    print("\nOptimal policy (sample of 10 states):")
    for i in range(10):
        print(f"State {states[i]} → Action {optimal_policy[i]}")


Iteration 1 completed.
Iteration 2 completed.
Iteration 3 completed.
Iteration 4 completed.
Iteration 5 completed.
Iteration 6 completed.
Iteration 7 completed.
Iteration 8 completed.
Iteration 9 completed.
Iteration 10 completed.
Iteration 11 completed.
Iteration 12 completed.
Iteration 13 completed.
Iteration 14 completed.
Iteration 15 completed.
Iteration 16 completed.
Iteration 17 completed.
Iteration 18 completed.
Iteration 19 completed.
Iteration 20 completed.
Iteration 21 completed.
Iteration 22 completed.
Iteration 23 completed.
Iteration 24 completed.
Iteration 25 completed.
Iteration 26 completed.
Iteration 27 completed.
Iteration 28 completed.
Iteration 29 completed.
Iteration 30 completed.
Iteration 31 completed.
Iteration 32 completed.
Iteration 33 completed.
Iteration 34 completed.
Iteration 35 completed.
Iteration 36 completed.
Iteration 37 completed.
Iteration 38 completed.
Iteration 39 completed.
Iteration 40 completed.
Iteration 41 completed.
Iteration 42 completed.
I